In [50]:
import pandas as pd
from backtesting.test import SMA
from backtesting.lib import SignalStrategy, TrailingStrategy


class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 10% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * 0.1
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [52]:
from backtesting import Backtest
from backtesting.test import EURUSD 

BTCUSDT = pd.read_csv("AAPL.csv", index_col=0, parse_dates=True, infer_datetime_format=True)
# print(BTCUSDT)
print(BTCUSDT)
bt = Backtest(BTCUSDT, SmaCross, cash=1000, commission=.004)

results = bt.run()
# print(results)
bt.plot()

                  Open        High         Low       Close    Volume
2021-08-30  149.000000  153.490005  148.610001  153.119995  90956700
2021-08-31  152.660004  152.800003  151.289993  151.830002  86453100
2021-09-01  152.830002  154.979996  152.339996  152.509995  80313700
2021-09-02  153.869995  154.720001  152.399994  153.649994  71115500
2021-09-03  153.759995  154.630005  153.089996  154.300003  57808700
...                ...         ...         ...         ...       ...
2022-08-23  167.080002  168.710007  166.649994  167.229996  54147100
2022-08-24  167.320007  168.110001  166.250000  167.529999  53841500
2022-08-25  168.779999  170.139999  168.350006  170.029999  51218200
2022-08-26  170.570007  171.050003  163.559998  163.619995  78823500
2022-08-29  161.149994  162.899994  159.820007  161.380005  73227700

[252 rows x 5 columns]


Row(id='20192', ...)